# Proceso ETL Dataset Steam Games

Vamos a proceder a extraer los conjuntos de datos que se encuentran comprimidos en el archivo ***steam_games.json.gz** para luego manipularlos y aplicar las transformaciones necesarias. El objetivo final es obtener un conjunto de datos limpio que pueda ser consumido por la API.

Importamos las librerias necesarias:

In [35]:
import pandas as pd
import gzip
import json

### Descomprimir datos

Descomprimimos el archivo y guardamos los datos en un dataframe.

In [36]:
data_games = []
with gzip.open('../Data/Data-Original/steam_games.json.gz', 'r') as f:
    for line in f:
        data_games.append(json.loads(line))

df_games = pd.DataFrame(data_games)
df_games.tail(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


Desanidamos las columnas genres, tags y specs

In [37]:
df_games = df_games.explode('genres')
df_games = df_games.explode('tags')
df_games = df_games.explode('specs')
df_games.reset_index(drop=True, inplace=True)
df_games.tail(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
2177133,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,VR,http://steamcommunity.com/app/681550/reviews/?...,Tracked Motion Controllers,4.99,True,681550,NaN
2177134,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,VR,http://steamcommunity.com/app/681550/reviews/?...,Standing,4.99,True,681550,NaN
2177135,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,VR,http://steamcommunity.com/app/681550/reviews/?...,Room-Scale,4.99,True,681550,NaN


Veamos la cantidad de filas duplicadas

In [38]:
df_games.duplicated().sum()

88309

### Tratamiento de Duplicados y Nulos

Eliminamos las filas duplicadas

In [39]:
df_games.drop_duplicates(inplace=True)
df_games.duplicated().sum()

0

In [40]:
df_games.isnull().sum()

publisher       341010
genres           86887
app_name            10
title            83046
url                  1
release_date     84408
tags               322
reviews_url        202
specs             3318
price            68777
early_access         1
id                 202
developer        93746
dtype: int64

Vamos a eliminar las filas que tengan títulos vacíos o nulos, ya que no aportarán información útil. Sin un título, no sabremos a qué juego se refiere la entrada.

In [41]:
df_games.dropna(subset=['title','id'], inplace=True)
df_games.isnull().sum()

publisher       257964
genres            3849
app_name             0
title                0
url                  0
release_date      1370
tags               312
reviews_url          0
specs             3316
price            65776
early_access         0
id                   0
developer        10700
dtype: int64

Eliminamos las columnas que representan urls ya que no aportan información a un modelo

In [42]:
df_games.drop(['url','reviews_url'], axis=1, inplace=True)
df_games.head(3)

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,Strategy,Single-player,4.99,False,761140,Kotoshiro
88311,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,Action,Single-player,4.99,False,761140,Kotoshiro
88312,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,Indie,Single-player,4.99,False,761140,Kotoshiro


### Transformacion de columnas

In [43]:
df_games['release_date'].unique()

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2016-11-19',
       'January 2018', '2018-10-01'], dtype=object)

Vamos a covertir a formato de fecha toda la columna **'release_date** que cumplan un formato iterpretado por la función to_datetime()

In [44]:
df_games['release_date'] = pd.to_datetime(df_games['release_date'], format='mixed', errors='coerce')
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2005581 entries, 88310 to 2177087
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   publisher     object        
 1   genres        object        
 2   app_name      object        
 3   title         object        
 4   release_date  datetime64[ns]
 5   tags          object        
 6   specs         object        
 7   price         object        
 8   early_access  object        
 9   id            object        
 10  developer     object        
dtypes: datetime64[ns](1), object(10)
memory usage: 183.6+ MB


Vamos a convertir a formato entero la columna **'id'**

In [45]:
df_games['id'] = df_games['id'].astype(int)

In [46]:
df_games['price'].unique()


array([4.99, 'Free To Play', 'Free to Play', 0.99, 3.99, 9.99, 18.99,
       29.99, nan, 10.99, 2.99, 1.59, 14.99, 1.99, 59.99, 8.99, 6.99,
       7.99, 39.99, 'Free', 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99, 179.0,
       26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 22.99, 320.0, 38.85,
       71.7, 995.0, 27.49, 3.39, 6.0, 19.95, 499.99, 199.99, 16.06, 4.68,
       131.4, 44.98, 2

Convertimos a valores numericos la columna precio.

In [47]:
df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce')
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2005581 entries, 88310 to 2177087
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   publisher     object        
 1   genres        object        
 2   app_name      object        
 3   title         object        
 4   release_date  datetime64[ns]
 5   tags          object        
 6   specs         object        
 7   price         float64       
 8   early_access  object        
 9   id            int32         
 10  developer     object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(8)
memory usage: 176.0+ MB


Guardamos la data extraida en un archivo csv

In [48]:
df_games.to_csv('../Data/Data-Limpia/steam_games.csv', sep=',', encoding='utf-8', index=False)